In [ ]:
import os
import time
import requests
import base64

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from dotenv import load_dotenv

load_dotenv()

email = os.getenv('EMAIL')
password = os.getenv('PASSWORD')
headless = os.getenv('HEADLESS')

captcha_str = os.getenv('CAPTCHA')

In [15]:
driver = webdriver.Chrome()

In [16]:
wait = WebDriverWait(driver,10)

In [17]:
driver.get("https://www.coursera.org/?authMode=login")
time.sleep(5)
email_field = driver.find_element(By.CSS_SELECTOR, 'input[name="email"]')
password_field = driver.find_element(By.CSS_SELECTOR, 'input[name="password"]')
email_field.send_keys(email)
password_field.send_keys(password)
password_field.send_keys(Keys.RETURN)

In [ ]:
url = input("Enter the URL of the course you want to download: ")

In [ ]:
driver.get(url)

In [19]:
time.sleep(5)

In [ ]:
links=[]
weeks = driver.find_elements(By.XPATH, '//a[@data-test="rc-WeekNavigationItem"]')
for week in weeks:
    link = week.get_attribute('href')
    links.append(link)

In [21]:
def get_quizzes():
    quizzes = []
    items = driver.find_elements(By.XPATH, '//a[@data-track-component="item_link"]')
    for item in items:
        link = item.get_attribute('href')
        if 'assignment-submission' in link:
            quizzes.append(link)
    return quizzes

In [22]:
def continue_button():
    print("Looking for continue button")
    try:
        button = driver.find_element(By.XPATH, "//button[span[text()='Continue']]")
        button.click()
        print("Clicking continue button")
    except:
        print("No continue button")

In [35]:
def get_answers():
    text = ""


    # Find all questions
    questions = driver.find_elements(By.XPATH, '//div[@role="group"]')
    
    for q in questions:
        text += q.get_attribute('innerText') + "\n"

        # Find correct answer
        try:
            correct = q.find_elements(By.XPATH, './/label[contains(@class, "cui-isChecked")]')
            for corr in correct:
                text += "Correct Answer: " + corr.get_attribute('innerText') + "\n"
        except:
            text += "Correct Answer: Not found\n"

        # Handle images in questions or answers
        images = q.find_elements(By.XPATH, './/img')
        for idx, img in enumerate(images):
            image_src = img.get_attribute('src')
            if image_src.startswith('data:image'):  # If already base64
                try:
                    header, base64_data = image_src.split(',', 1)
                    file_extension = header.split('/')[1].split(';')[0]
                    file_name = f"question_image_{idx}.{file_extension}"
                    with open(file_name, "wb") as image_file:
                        image_file.write(base64.b64decode(base64_data))
                    text += f"Image: {file_name}\n"
                except:
                    text += "Image: Unable to decode base64\n"
            else:
                try:
                    image_data = requests.get(image_src).content  # Download the image
                    file_extension = image_src.split('.')[-1]
                    file_name = f"question_image_{idx}.{file_extension}"
                    with open(file_name, "wb") as image_file:
                        image_file.write(image_data)
                    text += f"Image: {file_name}\n"
                except:
                    text += "Image: Unable to fetch\n"

    return text

In [44]:
text=""

In [45]:
for link in links:
    driver.get(link)
    time.sleep(10)
    quizzes = get_quizzes()
    for quiz in quizzes:
        driver.get(quiz)
        time.sleep(10)
        continue_button()

        # Wait for submission button to appear and click it
        WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//button[@data-testid="view-submission-button"]'))
        ).click()

        continue_button()
        time.sleep(10)
        text += get_answers()

Looking for continue button
No continue button
Looking for continue button
No continue button
Looking for continue button
No continue button
Looking for continue button
No continue button
Looking for continue button
No continue button
Looking for continue button
No continue button
Looking for continue button
No continue button
Looking for continue button
No continue button
Looking for continue button
No continue button
Looking for continue button
No continue button
Looking for continue button
No continue button
Looking for continue button
No continue button
Looking for continue button
No continue button
Looking for continue button
No continue button
Looking for continue button
No continue button
Looking for continue button
No continue button
Looking for continue button
No continue button
Looking for continue button
No continue button
Looking for continue button
No continue button
Looking for continue button
No continue button
Looking for continue button
No continue button
Looking for c